In [1]:
# Setup Notebook to load Django code
# From project root, run: jupyter-lab

import os
import sys
from pathlib import Path

django_project_dir = Path('../')
sys.path.insert(0, str(django_project_dir))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "ratom_api.settings.local")

import django
django.setup()

In [2]:
import pandas as pd

from django.db.models import Count
from django.contrib.postgres.search import SearchQuery, SearchVector, SearchRank

from ratom import models as ratom

# Entities (db model)

In [3]:
collection = ratom.Collection.objects.get(title="kate_symes")
collection

<Collection: kate_symes>

In [4]:
collection.message_set.count()

13615

In [5]:
ratom.Entity.objects.filter(message__collection=collection).count()

506087

In [6]:
entities = ratom.Entity.objects.filter(
    message__collection=collection
).values("label").annotate(
    count=Count("value")
).order_by("-count")
pd.DataFrame(list(entities.values("label", "count")))

,label,count
0,ORG,158786
1,PERSON,106988
2,CARDINAL,100218
3,DATE,44355
4,TIME,42828
5,WORK_OF_ART,16441
6,GPE,14856
7,MONEY,10056
8,ORDINAL,2034
9,PRODUCT,1925


In [7]:
person_entities = ratom.Entity.objects.filter(
    message__collection=collection,
    label="ORG"
).values("value").annotate(
    count=Count("value")
).order_by("-count")
pd.DataFrame(list(person_entities.values("value", "count")))[:40]

,value,count
0,PST,32233
1,"ZL Technologies, Inc.",13615
2,NSF,13615
3,Data Set,13615
4,"ZL Technologies,",13615
5,EML,13389
6,EDRM Enron Email Data Set,13105
7,Enron,4377
8,Prebon,988
9,EES,928


In [8]:
messages = collection.message_set.filter(
    entities__label="ORG",
    entities__value="Pinnacle West",
)
messages.count()

142

In [9]:
message = messages[5]
print(message.msg_body)

Body-Type: plain-text

Thanks so much! I'll pass this along to the traders.

Kate


Stephanie Piwetz   03/29/2001 11:52 AM

To: Kate Symes/PDX/ECT@ECT
cc:  

Subject: Re: APS/Pinnacle West  

Hello Kate,
APS and Pinnacle...We ran into a problem checking out for the end of the 
month, because a few deals had Pinnacle in the system, but Bloomberg on the 
bro confirm.  We had to ch them to match the broker confirms, because APS 
showed the deals as APS on there side.  They are trying to segregate 
companies little by little.  We have been told by Rhonda, our contract 
administrator that trading under both names are legally correct, both are 
under WSPP.  

DeeDee from Pinnacle said all trades are with Pinnacle unless Bloomberg 
trade; then cp should be APS.  DeeDee said shortly all trades will be under 
Pinnacle, but did not give me a specific date.       

Please let the traders know that all trades should be with Pinnacle West, and 
APS that are transacted with Bloomberg.  No, we will n

# PostgreSQL: Full Text Searching

In [18]:
collection = ratom.Collection.objects.get(title="kate_symes")
collection

<Collection: kate_symes>

In [19]:
collection.message_set.count()

13615

In [28]:
messages_filter = ratom.Message.objects.filter(
    collection=collection,
).exclude(
    directory__contains="/Deleted Items",
).filter(
    msg_tagged_body__icontains="Pinnacle West",
)
messages_filter.count()

138

In [29]:
search_query = SearchQuery('Pinnacle West')
search_vectors = (
    SearchVector('msg_subject', weight='A') +
    SearchVector('msg_body', weight='B') +
    SearchVector('msg_headers', weight='C')
)
search_rank = SearchRank(search_vectors, search_query)

messages = ratom.Message.objects.filter(
    collection=collection,
).exclude(
    directory__contains="/Deleted Items",
).annotate(
    search=search_vectors,
).filter(
    search=search_query,
).annotate(
    rank=search_rank,
).order_by(
    "-rank"
)

In [30]:
messages.count()

166

In [14]:
pd.DataFrame(
    list(
        messages.values(
            "msg_from",
            "sent_date",
            "msg_to",
            "msg_subject",
            "rank",
        )[:20],
    ),
)

,msg_from,sent_date,msg_to,msg_subject,rank
0,"""Kate Symes""",2001-03-30 00:47:00+00:00,"""Stephanie Piwetz""",Re: APS/Pinnacle West,0.999562
1,"""Kate Symes""",2001-03-30 00:47:00+00:00,"""Stephanie Piwetz""",Re: APS/Pinnacle West,0.999562
2,"""Kate Symes""",2001-03-30 00:47:00+00:00,"""Stephanie Piwetz""",Re: APS/Pinnacle West,0.999562
3,"""Kate Symes""",2001-03-30 00:47:00+00:00,"""Stephanie Piwetz""",Re: APS/Pinnacle West,0.999562
4,"""Stephanie Piwetz""",2001-03-30 02:52:00+00:00,"""Kate Symes""",Re: APS/Pinnacle West,0.999438
5,"""Stephanie Piwetz""",2001-03-30 02:52:00+00:00,"""Kate Symes""",Re: APS/Pinnacle West,0.999438
6,"""Stephanie Piwetz""",2001-03-30 02:52:00+00:00,"""Kate Symes""",Re: APS/Pinnacle West,0.999438
7,"""Rhonda L Denton""",2001-01-27 03:21:00+00:00,"""Kate Symes""",Re: Pinnacle West,0.998913
8,"""Rhonda L Denton""",2001-01-27 03:21:00+00:00,"""Kate Symes""",Re: Pinnacle West,0.998913
9,"""Rhonda L Denton""",2001-01-27 03:21:00+00:00,"""Kate Symes""",Re: Pinnacle West,0.998913


In [16]:
messages_without_person = messages.exclude(
    entities__label="PERSON",
)
messages_without_person.count()

0

In [16]:
pd.DataFrame(
    list(
        messages_without_person.values(
            "msg_from",
            "sent_date",
            "msg_to",
            "msg_subject",
            "rank",
        )[:20],
    ),
)

,msg_from,sent_date,msg_to,msg_subject,rank
0,"""Rhonda L Denton""",2001-01-27 02:19:00+00:00,"""Kate Symes""",Pinnacle West,0.995378
1,"""Rhonda L Denton""",2001-01-27 02:19:00+00:00,"""Kate Symes""",Pinnacle West,0.995378
2,"""Rhonda L Denton""",2001-01-27 02:19:00+00:00,"""Kate Symes""",Pinnacle West,0.995378
3,"""Kate Symes""",2000-12-11 21:26:00+00:00,"""Kimberly Hundl""",Re: Deal #477087.01 Pinnacle West...,0.991032
4,"""Kate Symes""",2000-12-11 21:26:00+00:00,"""Kimberly Hundl""",Re: Deal #477087.01 Pinnacle West...,0.991032
5,"""Kate Symes""",2000-12-11 21:26:00+00:00,"""Kimberly Hundl""",Re: Deal #477087.01 Pinnacle West...,0.991032
6,"""Kimberly Hundl""",2000-12-12 03:14:00+00:00,"""Kate Symes""",Re: Deal #477087.01 Pinnacle West...,0.991032
7,"""Kimberly Hundl""",2000-12-12 03:14:00+00:00,"""Kate Symes""",Re: Deal #477087.01 Pinnacle West...,0.991032
8,"""Kimberly Hundl""",2000-12-12 03:14:00+00:00,"""Kate Symes""",Re: Deal #477087.01 Pinnacle West...,0.991032
9,"""Kate Symes""",2000-12-11 21:26:00+00:00,"""Kimberly Hundl""",Re: Deal #477087.01 Pinnacle West...,0.991032


In [17]:
print(messages_without_person[0].msg_body)

Body-Type: plain-text

Please check deal no. 500771 under Pinnacle West.  I thought Arizona Public 
Service kept all transmisson.  Please let me know what you find out.

***********
EDRM Enron Email Data Set has been produced in EML, PST and NSF format by ZL Technologies, Inc. This Data Set is licensed under a Creative Commons Attribution 3.0 United States License <http://creativecommons.org/licenses/by/3.0/us/> . To provide attribution, please cite to "ZL Technologies, Inc. (http://www.zlti.com)."
***********


# Using MessageManager

In [3]:
collection = ratom.Collection.objects.get(title="kate_symes")
collection

<Collection: kate_symes>

In [4]:
text = 'Pinnacle West'

In [8]:
messages = ratom.Message.objects.search(
    text,
).filter(
    collection=collection,
)
messages.count()

166

# JSONField .data Field

In [3]:
collection = ratom.Collection.objects.get(title="susan_bailey")
collection

<Collection: susan_bailey>

In [4]:
collection.message_set.count()

2270

In [8]:
messages = ratom.Message.objects.filter(
    collection=collection,
).filter(
    data__labels__contains="PERSON"
)
messages.count()

1036